### Problema

Se desea diseñar un intercambiador de calor de tubos y coraza, con queroseno y petróleo como fluidos de trabajo. La caída de presión en el lado coraza debe ser de 15kPa y 40 kPa para el lado tubos. 

Considerar que el material del intercambiador es acero al carbón (k = 43W/mK). 

#### Consideraciones
<ul>

<li>El sistema se encuentra bien aislado.</li>
<li>El flujo es contracorriente.</li>
</ul>

#### Propiedades

<table title = "Propiedades">
<tr>
    <th scope = "col"> </th>
    <th scope = "col"> Queronseno </th>
    <th scope = "col"> Petróleo </th>
</tr>
<tr>
    <td> Cp [J/kg°C]</td>
    <td> 2470 </td>
    <td> 2050 </td>
</tr>
<tr>
    <td> k [W/m°C] </td>
    <td> 0.132 </td>
    <td> 0.133 </td>
</tr>
<tr>
    <td> μ [mPa s]</td>
    <td> 0.4 </td>
    <td> 3.6 </td>
</tr>
<tr>
    <td> ρ [kg/m3] </td>
    <td> 730 </td>
    <td> 830 </td>
</tr>
<tr>
    <td> m [kg/s] </td>
    <td> 25 </td>
    <td> ¿? </td>
</tr>
<tr>
    <td> Ti [°C] </td>
    <td> 200 </td>
    <td> 35 </td>
</tr>
<tr>
    <td> To [°C] </td>
    <td> 95.16 </td>
    <td> 74.94 </td>
</tr>
<tr>
    <td> R [m2°C/W] </td>
    <td> 0.0002 </td>
    <td> 0.0005 </td>
</tr>
</table>



#### Anexo

<ul>
<li>Gráficos de jh y jf en función de Re para tubos y para coraza, tomado de [1]. </li>

<img src = "./Graficos/jh lado tubos.png" alt = "Gráfico jh lado tubos">
<p> Fig. 1. Valores de jh en función de Re y la relación L/D para lado tubos</p>
<img src = "./Graficos/jf lado tubos.png" alt = "Gráfico jf lado tubos">
<p> Fig. 2. Valores de jf para lado tubos </p>
<img src = "./Graficos/jh lado coraza.png" alt = "Gráfico jh lado coraza">
<p> Fig. 3. Valores de jh para lado coraza </p>
<img src = "./Graficos/jf lado coraza.png" alt = "Gráfico jf lado coraza">
<p> Fig. 4. Valores de jf para lado coraza </p>

<li> Valores típicos de U para intercambiador de tubos y coraza, tomado de [1].</li>
<img src = "./Graficos/Valores_tipicos_U_ITyC.png" alt = "Tabla de valores de U para varios sistemas, ITyC.">
<p> Fig. 5. Tabla de valores de U para varios sistemas, ITyC. </p>


In [1]:
import objetos as obj
import Diseño_tubos_coraza as dtc
import matplotlib.pyplot as plt

Se crean objetos con referencia a los fluidos y el material.

In [2]:
# creación de objetos

tuberia_acero = obj.Tuberia(
    material = "Acero al carbón",
    do = 0.019,
    espesor = 0.001,
    L = 8.61,
    k = 43 
)

FlujoCaliente = obj.FlujoFluido(
    fluido = "queroseno",
    temperatura_entrada= 200,
    temperatura_salida= 95.16,
)

FlujoFrio = obj.FlujoFluido(
    fluido = "petróleo",
    temperatura_entrada= 35,
    temperatura_salida= 74.9,
)

propiedadesQueroseno = obj.Propiedades(
    fluido = "queroseno",
    temp_entrada_salida = [200,95.16],
    temperaturas_propiedades = "NA",
    densidad = 730,
    viscosidad = 0.4e-3,
    cp = 2470,
    coeficiente_conductivo = 0.132,
    numero_prandtl = 0.4e-3*2470/0.132,
    Rf = 0.0002,
    flujo_masico = 25
)

propiedadesPetroleo = obj.Propiedades(
    fluido = "Petroleo",
    temp_entrada_salida = [35,74.94],
    temperaturas_propiedades = "NA",
    densidad = 830,
    viscosidad = 3.6e-3,
    cp = 2050,
    coeficiente_conductivo = 0.133,
    numero_prandtl = 3.6e-3*2050/0.133,
    Rf = 0.0005,
    flujo_masico = 0
)

intercambiador1 = obj.Intercambiador(

)


Se determinan los parámetros faltantes, como es Q y el flujo másico del petróleo por medio de un balance de energía

In [3]:
Q, m2 = obj.balance_energia(
    m = [25,0],
    Cp = [2470,2050],
    Ti = [200,35],
    To = [95.16,74.94],
    calc = "m"
)

print("Q = ",Q ,"\nm2 = ", m2)

propiedadesPetroleo.flujo_masico = m2

Se desconoce el flujo másico del fluido que se calienta.
Q =  6473870.0 
m2 =  79.0682365010931


Con base en la resistencia de ensuciamiento se decide que el queroseno, que es el que menos ensucia, es el fluido que va por coraza; mientras que el petróleo va por tubos.


In [4]:
FlujoTubos = propiedadesPetroleo
FlujoCoraza = propiedadesQueroseno

## Iteración 1

Se procede ahora a proponer un coeficiente global de transferencia de calor para el intercambiador, el cual, con base en la figura 5 y teniendo en cuenta que son aceites pesados (Heavy oils) los fluidos de trabajo, se escoge del promedio entre 50 y 300 W/m2°C.

Además, se establecen características del diseño, como el número de pasos por tubos y por coraza, el corte de bafle y el tipo de pitch.

In [5]:
U = 291#(50+300)/2

pasos_tubos = 1
pasos_coraza = 1
corte_bafle = '15%'

tipo_pitch = "triangular"

Se evalúan las porpiedades para el lado tubos

In [6]:
obj.lado_tubos(
    Q = Q,
    U = U, 
    pasos_tubos = pasos_tubos,
    caracteristicas_tuberia = tuberia_acero, 
    FlujoCaliente = FlujoCaliente, 
    FlujoFrio = FlujoFrio, 
    FlujoTubos = FlujoTubos,
    Intercambiador = intercambiador1
    )


Para el lado tubos:


(226.5690672072488, 541.6321489955797, 7084.4857058898515)

Para el lado coraza

In [7]:
obj.lado_coraza(
    caracteristicas_tuberia = tuberia_acero,
    FlujoCoraza = FlujoCoraza,
    numero_tubos = intercambiador1.numero_tubos,
    pasos_tubos = pasos_tubos,
    tipo_pitch = tipo_pitch,
    corte_bafle = corte_bafle,
    Intercambiador = intercambiador1
)

(2002.815051782778, 245205.3155494682)

In [8]:
print(intercambiador1)


LADO TUBOS: 


    Pasos: 1
    Diferencia de temperaturas media: 79.92081901047641
    Área total de transferencia de calor: 278.36271225024535 m2
    Número de tubos: 541.6321489955797 tubos
    Longitud tubos: 8.61 m
    Diámetro exterior: 0.019 m
    Espesor: 0.001 m 
    Tubos por paso: 541.6321489955797 tubos
    velocidad en lado tubos: 0.7748752719680598 m/s
    Reynolds en tubos: 3037.0805798525903

    jh: 0.0025
    jf: 0.0064
    h lado tubos (hi): 226.5690672072488 W/m2K
    ΔP lado tubos: 7084.4857058898515 Pa

LADO CORAZA:

    Diámetro del banco de tubos: 0.6118568737572134 m
    Diámetro interno coraza: 0.6732323606103877 m
    Separación de bafles: 0.2692929442441551 m
    Pitch de los tubos: 0.02375 m
    Tipo de pitch: triangular
    Área de flujo en coraza: 0.03625934490984282 m2
    Diámetro hidráulico: 0.01349095 m
    Velocidad lado coraza: 0.9444896323311487 m/s
    Re: 23254.263889668688
    Corte de bafle: 15%

    jh: 0.0045
    jf: 0.059
    h lado coraza 

Con la información obtenida, se calcula el coeficiente global de transferencia de calor y se determina el error asociado.


In [9]:
U_calculado = obj.coefiente_global_U(
    h_conv = [intercambiador1.hi,intercambiador1.h_coraza],
    k = tuberia_acero.k,
    di = tuberia_acero.di,
    do = tuberia_acero.do,
    foul_f = [propiedadesPetroleo.Rf, propiedadesQueroseno.Rf]
)

error_U = obj.error(
    valor_propuesto=U,
    valor_calculado = U_calculado
    )

error_presion_tubos = obj.error(
    valor_propuesto = 40e3,
    valor_calculado = intercambiador1.caida_presion_tubos
)

error_presion_coraza = obj.error(
    valor_propuesto = 15e3,
    valor_calculado = intercambiador1.caida_presion_coraza
)

El coeficiente global de transferecia de calor: 160.88515785005987 W/m2K
Para el valor calculado = 160.88515785005987, %error = 44.71300417523716%
Para el valor calculado = 7084.4857058898515, %error = 82.28878573527538%
Para el valor calculado = 245205.3155494682, %error = 1534.7021036631213%


Podemos observar que los valores obtenidos se salen del rango de aceptación (%e para U = 30%, %e ΔP = 10%). Como el coeficiente global de transferencia de calor U se sale del rango de valores típicos, se procede a redimensionar el equipo proponiendo ahora 2 pasos por tubos.

## Iteración 2

In [10]:
pasos_tubos = 2
corte_bafle = "45%"

intercambiador2 = obj.Intercambiador()

In [11]:
obj.lado_tubos(
    Q = Q,
    U = U, 
    pasos_tubos = pasos_tubos,
    caracteristicas_tuberia = tuberia_acero, 
    FlujoCaliente = FlujoCaliente, 
    FlujoFrio = FlujoFrio, 
    FlujoTubos = FlujoTubos,
    Intercambiador = intercambiador2
    )

obj.lado_coraza(
    caracteristicas_tuberia = tuberia_acero,
    FlujoCoraza = FlujoCoraza,
    numero_tubos = intercambiador2.numero_tubos,
    pasos_tubos = pasos_tubos,
    tipo_pitch = tipo_pitch,
    corte_bafle = corte_bafle,
    Intercambiador = intercambiador2
)

print(intercambiador2)

Para el lado tubos:

LADO TUBOS: 


    Pasos: 2
    Diferencia de temperaturas media: 79.92081901047641
    Área total de transferencia de calor: 278.36271225024535 m2
    Número de tubos: 541.6321489955797 tubos
    Longitud tubos: 8.61 m
    Diámetro exterior: 0.019 m
    Espesor: 0.001 m 
    Tubos por paso: 270.8160744977898 tubos
    velocidad en lado tubos: 1.5497505439361197 m/s
    Reynolds en tubos: 6074.161159705181

    jh: 0.0035
    jf: 0.0055
    h lado tubos (hi): 634.3933881802967 W/m2K
    ΔP lado tubos: 49406.65558778859 Pa

LADO CORAZA:

    Diámetro del banco de tubos: 0.6180017537735323 m
    Diámetro interno coraza: 0.6795469622127572 m
    Separación de bafles: 0.2718187848851029 m
    Pitch de los tubos: 0.02375 m
    Tipo de pitch: triangular
    Área de flujo en coraza: 0.03694272590820693 m2
    Diámetro hidráulico: 0.01349095 m
    Velocidad lado coraza: 0.9270180935635229 m/s
    Re: 22824.097417583478
    Corte de bafle: 45%

    jh: 0.003
    jf: 0.028
 

In [12]:
U_calculado2 = obj.coefiente_global_U(
    h_conv = [intercambiador2.hi,intercambiador2.h_coraza],
    k = tuberia_acero.k,
    di = tuberia_acero.di,
    do = tuberia_acero.do,
    foul_f = [propiedadesPetroleo.Rf, propiedadesQueroseno.Rf]
)

error_U = obj.error(
    valor_propuesto=U,
    valor_calculado = U_calculado2
    )

error_presion_tubos = obj.error(
    valor_propuesto = 40e3,
    valor_calculado = intercambiador2.caida_presion_tubos
)

error_presion_coraza = obj.error(
    valor_propuesto = 15e3,
    valor_calculado = intercambiador2.caida_presion_coraza
)

El coeficiente global de transferecia de calor: 302.2778158976255 W/m2K
Para el valor calculado = 302.2778158976255, %error = 3.8755381091496517%
Para el valor calculado = 49406.65558778859, %error = 23.51663896947148%
Para el valor calculado = 112103.17987557504, %error = 647.3545325038335%


Por los resultados obtenidos, se cambian las características del intercambiador.

## Iteración 3

In [13]:
tuberia_acero2 = obj.Tuberia(
    material = "Acero al carbón",
    do = 0.020,
    espesor = 0.001,
    L = 6,
    k = 43
)

intercambiador3 = obj.Intercambiador()


In [14]:
obj.lado_tubos(
    Q = Q,
    U = U, 
    pasos_tubos = pasos_tubos,
    caracteristicas_tuberia = tuberia_acero2, 
    FlujoCaliente = FlujoCaliente, 
    FlujoFrio = FlujoFrio, 
    FlujoTubos = FlujoTubos,
    Intercambiador = intercambiador3
    )

obj.lado_coraza(
    caracteristicas_tuberia = tuberia_acero2,
    FlujoCoraza = FlujoCoraza,
    numero_tubos = intercambiador3.numero_tubos,
    pasos_tubos = pasos_tubos,
    tipo_pitch = tipo_pitch,
    corte_bafle = corte_bafle,
    Intercambiador = intercambiador3
)

print(intercambiador3)



Para el lado tubos:

LADO TUBOS: 


    Pasos: 2
    Diferencia de temperaturas media: 79.92081901047641
    Área total de transferencia de calor: 278.36271225024535 m2
    Número de tubos: 738.3800271182239 tubos
    Longitud tubos: 6 m
    Diámetro exterior: 0.02 m
    Espesor: 0.001 m 
    Tubos por paso: 369.19001355911195 tubos
    velocidad en lado tubos: 1.0140027285864588 m/s
    Reynolds en tubos: 4208.111323633804

    jh: 0.00337
    jf: 0.0065
    h lado tubos (hi): 399.66657678551553 W/m2K
    ΔP lado tubos: 16925.91091196233 Pa

LADO CORAZA:

    Diámetro del banco de tubos: 0.7485878436101264 m
    Diámetro interno coraza: 0.8137398398506381 m
    Separación de bafles: 0.3254959359402553 m
    Pitch de los tubos: 0.025 m
    Tipo de pitch: triangular
    Área de flujo en coraza: 0.052973802156811384 m2
    Diámetro hidráulico: 0.014201000000000005 m
    Velocidad lado coraza: 0.6464813539547363 m/s
    Re: 16754.744116207963
    Corte de bafle: 45%

    jh: 0.0035
    jf

In [15]:
U_calculado3 = obj.coefiente_global_U(
    h_conv = [intercambiador3.hi,intercambiador3.h_coraza],
    k = tuberia_acero2.k,
    di = tuberia_acero2.di,
    do = tuberia_acero2.do,
    foul_f = [propiedadesPetroleo.Rf, propiedadesQueroseno.Rf]
)

error_U = obj.error(
    valor_propuesto=U,
    valor_calculado = U_calculado3
    )

error_presion_tubos = obj.error(
    valor_propuesto = 40e3,
    valor_calculado = intercambiador3.caida_presion_tubos
)

error_presion_coraza = obj.error(
    valor_propuesto = 15e3,
    valor_calculado = intercambiador3.caida_presion_coraza
)

El coeficiente global de transferecia de calor: 222.31965571224183 W/m2K
Para el valor calculado = 222.31965571224183, %error = 23.601492882391124%
Para el valor calculado = 16925.91091196233, %error = 57.685222720094174%
Para el valor calculado = 38671.27084632417, %error = 157.80847230882782%


## Iteración 4

In [16]:
tuberia_acero3 = obj.Tuberia(
    material = "Acero al carbón",
    do = 0.020,
    espesor = 0.002,
    L = 4,
    k = 43
)

intercambiador4 = obj.Intercambiador()
pasos_tubos = 4
corte_bafle = "15%"

obj.lado_tubos(
    Q = Q,
    U = U_calculado3, 
    pasos_tubos = pasos_tubos,
    caracteristicas_tuberia = tuberia_acero3, 
    FlujoCaliente = FlujoCaliente, 
    FlujoFrio = FlujoFrio, 
    FlujoTubos = FlujoTubos,
    Intercambiador = intercambiador4
    )

obj.lado_coraza(
    caracteristicas_tuberia = tuberia_acero3,
    FlujoCoraza = FlujoCoraza,
    numero_tubos = intercambiador4.numero_tubos,
    pasos_tubos = pasos_tubos,
    tipo_pitch = tipo_pitch,
    corte_bafle = corte_bafle,
    Intercambiador = intercambiador4
)

print(intercambiador4)


Para el lado tubos:

LADO TUBOS: 


    Pasos: 4
    Diferencia de temperaturas media: 79.92081901047641
    Área total de transferencia de calor: 364.35621945037497 m2
    Número de tubos: 1449.7273342950639 tubos
    Longitud tubos: 4 m
    Diámetro exterior: 0.02 m
    Espesor: 0.002 m 
    Tubos por paso: 362.43183357376597 tubos
    velocidad en lado tubos: 1.3072774726775374 m/s
    Reynolds en tubos: 4822.401343654916

    jh: 0.0038
    jf: 0.0058
    h lado tubos (hi): 581.0054151763774 W/m2K
    ΔP lado tubos: 40000.25458568539 Pa

LADO CORAZA:

    Diámetro del banco de tubos: 1.0370494920596192 m
    Diámetro interno coraza: 1.1101687990303057 m
    Separación de bafles: 0.4440675196121223 m
    Pitch de los tubos: 0.025 m
    Tipo de pitch: triangular
    Área de flujo en coraza: 0.09859798098723133 m2
    Diámetro hidráulico: 0.014201000000000005 m
    Velocidad lado coraza: 0.34733546264908577 m/s
    Re: 9001.832401770394
    Corte de bafle: 15%

    jh: 0.0046
    jf: 

In [17]:
U_calculado4 = obj.coefiente_global_U(
    h_conv = [intercambiador4.hi,intercambiador4.h_coraza],
    k = tuberia_acero3.k,
    di = tuberia_acero3.di,
    do = tuberia_acero3.do,
    foul_f = [propiedadesPetroleo.Rf, propiedadesQueroseno.Rf]
)

error_U = obj.error(
    valor_propuesto=U_calculado3,
    valor_calculado = U_calculado4
    )

error_presion_tubos = obj.error(
    valor_propuesto = 40e3,
    valor_calculado = intercambiador4.caida_presion_tubos
)

error_presion_coraza = obj.error(
    valor_propuesto = 15e3,
    valor_calculado = intercambiador4.caida_presion_coraza
)

El coeficiente global de transferecia de calor: 229.54037838168887 W/m2K
Para el valor calculado = 229.54037838168887, %error = 3.2479011566989575%
Para el valor calculado = 40000.25458568539, %error = 0.0006364642134758469%
Para el valor calculado = 8434.145606903465, %error = 43.77236262064357%


## Iteración 5

In [18]:
intercambiador5 = obj.Intercambiador()
corte_bafle = "20%"

obj.lado_tubos(
    Q = Q,
    U = U_calculado3, 
    pasos_tubos = pasos_tubos,
    caracteristicas_tuberia = tuberia_acero3, 
    FlujoCaliente = FlujoCaliente, 
    FlujoFrio = FlujoFrio, 
    FlujoTubos = FlujoTubos,
    Intercambiador = intercambiador5
    )

obj.lado_coraza(
    caracteristicas_tuberia = tuberia_acero3,
    FlujoCoraza = FlujoCoraza,
    numero_tubos = intercambiador5.numero_tubos,
    pasos_tubos = pasos_tubos,
    tipo_pitch = tipo_pitch,
    corte_bafle = corte_bafle,
    Intercambiador = intercambiador5
)

print(intercambiador5)

Para el lado tubos:

LADO TUBOS: 


    Pasos: 4
    Diferencia de temperaturas media: 79.92081901047641
    Área total de transferencia de calor: 364.35621945037497 m2
    Número de tubos: 1449.7273342950639 tubos
    Longitud tubos: 4 m
    Diámetro exterior: 0.02 m
    Espesor: 0.002 m 
    Tubos por paso: 362.43183357376597 tubos
    velocidad en lado tubos: 1.3072774726775374 m/s
    Reynolds en tubos: 4822.401343654916

    jh: 0.0038
    jf: 0.0058
    h lado tubos (hi): 581.0054151763774 W/m2K
    ΔP lado tubos: 40000.25458568539 Pa

LADO CORAZA:

    Diámetro del banco de tubos: 1.0370494920596192 m
    Diámetro interno coraza: 1.1101687990303057 m
    Separación de bafles: 0.4440675196121223 m
    Pitch de los tubos: 0.025 m
    Tipo de pitch: triangular
    Área de flujo en coraza: 0.09859798098723133 m2
    Diámetro hidráulico: 0.014201000000000005 m
    Velocidad lado coraza: 0.34733546264908577 m/s
    Re: 9001.832401770394
    Corte de bafle: 20%

    jh: 0.00645
    jf:

In [19]:
U_calculado5 = obj.coefiente_global_U(
    h_conv = [intercambiador5.hi,intercambiador5.h_coraza],
    k = tuberia_acero3.k,
    di = tuberia_acero3.di,
    do = tuberia_acero3.do,
    foul_f = [propiedadesPetroleo.Rf, propiedadesQueroseno.Rf]
)

error_U = obj.error(
    valor_propuesto=U_calculado3,
    valor_calculado = U_calculado5
    )

error_presion_tubos = obj.error(
    valor_propuesto = 40e3,
    valor_calculado = intercambiador5.caida_presion_tubos
)

error_presion_coraza = obj.error(
    valor_propuesto = 15e3,
    valor_calculado = intercambiador5.caida_presion_coraza
)

El coeficiente global de transferecia de calor: 251.53582912248396 W/m2K
Para el valor calculado = 251.53582912248396, %error = 13.141516127596889%
Para el valor calculado = 40000.25458568539, %error = 0.0006364642134758469%
Para el valor calculado = 14883.78636512376, %error = 0.7747575658415977%


Esta última iteración satisface los requerimentos en caída de presión y de transferencia de calor, entonces este es un diseño que se ajusta al servicio requerido.

### Referencias

[1] Sinnott, R. (2005). Chemical Engineering Design: Chemical Engineering Volume 6 (Chemical Engineering Series) (4th ed.). Butterworth-Heinemann.